## 什么是工具（Tools）？

Tools 是一种基于 Python 函数的抽象形式，包含有函数名称、描述以及期望参数的结构 `schema`。

换句话说，tools 是人为进行定义，且输入输出符合 LLM 能力范围，并且可以让 LLM 自行决定是否需要调用的一类函数体。使用注解 `@tool` 标记 tools，有若干好处：
* 可以定制或者自动化推断工具的名称、描述和预期参数；
* 可以定义工具来返回一些有用的 output（比如图像、dataframes 等）；
* 从模型中隐藏输入的参数，隐式进行处理注入的参数。

In [1]:
from typing import Annotated

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,  # 此时需要我们的 LLM 更严谨
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

创建 LLM tool，具体的参数可以参考[官方文档](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.convert.tool.html)。这里我们先使用注解进行定义一个简单的函数。

In [5]:
from langchain_core.tools import tool


# 这是一类被称为 Creating tools from functions 的用法
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


print(multiply.name)
print(multiply.description)  # 必须需要有注释才能返回 description
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


有非常多的方式可以创建 tool，在[官方文档](https://python.langchain.com/docs/how_to/custom_tools/)中还提供了更为复杂的、与别的包相互集成的用法，比如与 annotations 一起结合使用：
```python
from typing import Annotated, List

@tool
def multiply_by_max(
    a: Annotated[int, "scale factor"],
    b: Annotated[List[int], "list of ints over which to take maximum"],
) -> int:
    """Multiply a by the maximum of b."""
    return a * max(b)
```
或者是与 pydantic 结合，传入 JSON 格式数据：
```python
from pydantic import BaseModel, Field

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

@tool("multiplication-tool", args_schema=CalculatorInput, return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b
```
不加以尝试，当做了解即可。

In [8]:
# 或者是使用类方法来初始化 tools
from langchain_core.tools import StructuredTool


def mult(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amult(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=mult, coroutine=amult)

In [7]:
# 创建好 tool 之后，我们可以调用它们得到结果
print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 5}))

6
10


## InjectedToolArg
接下来我们讨论一下 `InjectedToolArg` 这一个 annotation，它用于告诉 LangChain：“这个参数不应该由语言模型来填充，而是我（程序员）在 Runtime 时主动传进去的。”

### ❓为什么需要一个这样的函数？
因为有一些工具类再进行操作的时候，需要带上一些 Runtime 的参数（比如当前登录着的用户、Session ID等等），这一些参数是只有后端才会知道，但不能/没必要传给 LLM，LLM 不应该也不会知道它。类比一下，比如 LLM 只需要关注合同上的条款，但不需要知道签合同的甲方乙方。因此有些函数需要“隐藏”掉一些 `schema`。

### 如何在运行时传入参数给 tools 呢？
参考官方文档[How to pass run time values to tools](https://python.langchain.com/docs/how_to/tool_runtime/)，

In [13]:
from langchain_core.tools import tool, InjectedToolArg


# 一个简单的 Runtime 案例
@tool
def greet(user_id: InjectedToolArg, data: str) -> str:
    """在用户登录的时候打招呼，然后做一个用户不在线时期的消息汇总。"""
    return f"Hello User {user_id}! Here is the data: {data}"


print('function name:', greet.name)
print('function desc:', greet.description)  # 必须需要有注释才能返回 description
# print('function args:',greet.args) # <--- 会报错，因为 InjectedToolArg 是一个特殊的内部类型，Pydantic 并不知道如何为它生成 schema。从侧面说明了这是一个 Runtime 参数，在静态编译的时候被“隐藏”掉了，不是一个有效的参数类型。

function name: greet
function desc: 在用户登录的时候打招呼，然后做一个用户不在线时期的消息汇总。


In [27]:
from typing import Annotated


# 修改一下我们刚刚的参数，我们使用 Annotated 来增强代码的鲁棒性，声明这个参数是 str 类型，但有额外元信息，
# LangChain 会读取这个元信息并将其标记为“运行时注入的参数”。
@tool
def greet(
        user_id: Annotated[str, InjectedToolArg],  # <--- 注意此处的写法
        data: str
) -> str:
    """在用户登录的时候打招呼，然后做一个用户不在线时期的消息汇总。"""
    return f"Hello User {user_id}! Here is the data: {data}"


print('Input schemas:', greet.get_input_schema().model_json_schema()['required'])
print('Tool call schema:', greet.tool_call_schema.model_json_schema()['required'])  # <--- 可以看到调用时不需要传入 user_id

Input schemas: ['user_id', 'data']
Tool call schema: ['data']


## RunnableConfig
官方文档在[这里](https://python.langchain.com/docs/concepts/tools/#runnableconfig)，从名字可以看出来，与上面那个相似，`RunnableConfig` 在运行时将值传递给 tools。

### 与 `InjectedToolArg` 的区别

| 特性               | `InjectedToolArg`                 | `RunnableConfig`                   |
|------------------|-----------------------------------|------------------------------------|
| 传参方式             | 用函数参数注解 `Annotated`               | 用 `config={...}` 注入                |
| 参数作用范围           | 单一参数                              | 全局上下文、可访问多字段                       |
| 是否出现在工具 schema 中 | ❌ 否                               | ❌ 否                                |
| 获取方式             | 直接作为函数参数                          | 通过函数参数 `config: RunnableConfig` 获取 |
| 是否自动注入           | 否，需要显式 `.invoke(..., config=...)` | 是，框架在运行时自动传入                       |


In [34]:
# 一个简单的例子
from langchain_core.runnables import RunnableConfig


@tool
def secure_action(data: str, config: RunnableConfig) -> str:
    """执行一个需要用户身份的动作。"""
    user_id = config['configurable'].get("user_id", "anonymous")
    return f"[{user_id}] Upload the data: {data}"


response = secure_action.invoke(
    {"data": "Hello, how are you?"},
    config={"configurable": {"user_id": "Nethan123"}}  # <--- 如果注释掉这句话，用户名就会是 anonymous
)

print(response)

[Nethan123] Upload the data: Hello, how are you?


🤔为什么 LangChain 运行的时候，模型不会知道我的 `RunnableConfig`？

因为 `RunnableConfig` 是在程序运行时手动注入的，它不会被包含在 Prompt 或工具的 schema 中，所以模型完全“看不到”它。

**深层原因**：LangChain 的 `RunnableConfig` 和 LLM 的“输入”是分开的两层系统。在 LangChain 中，调用链是这样分开的：
1. LLM 可见的内容（模型输入）：
    * Prompt
    * Tool 的参数 schema（函数名 + 参数名 + docstring）

2. LLM 不可见的内容（LangChain `RunnableConfig`）：
    * config={"configurable": {...}}。这些值不会出现在 prompt，也不会出现在 function schema 中。

In [41]:
@tool
def secure_action(content: str, config: RunnableConfig) -> str:
    """帮用户看一看合同条款，找出有纰漏的地方并提出修改建议。"""
    user_a = config['configurable'].get("user_a", "anonymous")
    user_b = config['configurable'].get("user_b", "anonymous")
    return f"The contract of [{user_a} and {user_b}] has content of {content}..."


# 我们调用的方式为如下
action = secure_action.invoke(
    {"content": "Contract content: ******"},
    config={
        "configurable": {"user_a": "Alice", "user_b": "Bob"}
    }
)

print("In the view of programmer:", action)
print("===============================================================")
print("In the view of LLM:", 'secure_action(content="Contract content: ******")')
print(secure_action.tool_call_schema.model_json_schema())

In the view of programmer: The contract of [Alice and Bob] has content of Contract content: ******...
In the view of LLM: secure_action(content="Contract content: ******")
{'description': '帮用户看一看合同条款，找出有纰漏的地方并提出修改建议。', 'properties': {'content': {'title': 'Content', 'type': 'string'}}, 'required': ['content'], 'title': 'secure_action', 'type': 'object'}


我们已经接近 LangChain Runtime injection 的核心问题了：

在后端人员看来，是知道传入了什么参数，所以 `action` 在后端看来是：
```text
The contract of [Alice and Bob] has content of Contract content: ******...
```
但是在 LLM 看来，它并不知道我们究竟传入了哪些参数，这些参数对它是隐藏的，因此在它看来，它只需要完成这样一个函数的 schema 即可（即根据 content 修改合同的内容。）
```json
{
  "name": "secure_action",
  "description": "帮用户看一看合同条款，找出有纰漏的地方并提出修改建议。",
  "parameters": {
    "content": {
      "type": "string",
      "description": ""
    }
  }
}
```

因此我们引入官方文档的一句话：LangChain 通过 `RunnableConfig` 和 `InjectedToolArg`，将“模型能看到的输入”和“程序员注入的控制参数”分离开来，实现更安全、更灵活、更可控的 Tool 调用方式。

官方文档内还有 [InjectedState](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.tool_node.InjectedState) 和 [InjectedStore](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.tool_node.InjectedStore) 两个高级功能，后续继续补充。对于学习到这里而言，已经是掌握 tools 定义的基本用法了。